In [1]:
import sys
sys.path.insert(0, '../' )

In [2]:
from libgf264fft import clib_wrapper as cl

cl.gf264_mul( cl.index_to_gf264(1) , cl.index_to_gf264(1) )

1

In [3]:
from utils import randombytes as rd

rd.randombytes(16)


b'\xb5\x1f\xac\x88\xdd\xb4\xdb\x1f\x82U\xf4\xb9\xcf\x05\x15\xf2'

In [4]:
from aes128 import aes128_encrypt

#pt = _rand_bytes(16)
#sk = _rand_bytes(16)
pt = bytes( [i*16+i for i in range(16)] )
sk = bytes( [i for i in range(16)] )

ct, rd_states , rk , rk_tmp = aes128_encrypt( pt , sk )

print( sk )
print( pt )
print( rd_states , rk , rk_tmp  )
print( "ct0", ct )


from Crypto.Cipher import AES
ct2 = AES.new( sk , AES.MODE_ECB ).encrypt( pt )
print( "ct2" , ct2 )


b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x00\x11"3DUfw\x88\x99\xaa\xbb\xcc\xdd\xee\xff'
b'_rd\x15W\xf5\xbc\x92\xf7\xbe;)\x1d\xb9\xf9\x1a\xff\x87\x96\x841\xd8jQdQQ\xfaw:\xd0\tL\x9c\x1ef\xf7q\xf0v,?\x86\x8eSM\xf2Vc\x85\xb7\x9f\xfcS\x8d\xf9\x97\xbeG\x8euG\xd6\x91\xf4\xbc\xd4T2\xe5T\xd0u\xf1\xd6\xc5\x1d\xd0;<\x98\x16\xeet\x00\xf8\x7fUk,\x04\x9c\x8eZ\xd06\xc5~\x1c\x15\x9a\x9b\xd2\x86\xf0_K\xe0\x98\xc649\xba\xa0=\xe7\xa1\xf9\xb5n\xd5Q,\xba_AM#\xe9\xf7N\xec\x020 \xf6\x1b\xf2\xcc\xf25<!\xc7' b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\xd6\xaat\xfd\xd2\xafr\xfa\xda\xa6x\xf1\xd6\xabv\xfe\xb6\x92\xcf\x0bd=\xbd\xf1\xbe\x9b\xc5\x00h0\xb3\xfe\xb6\xfftN\xd2\xc2\xc9\xbflY\x0c\xbf\x04i\xbfAG\xf7\xf7\xbc\x955>\x03\xf9l2\xbc\xfd\x05\x8d\xfd<\xaa\xa3\xe8\xa9\x9f\x9d\xebP\xf3\xafW\xad\xf6"\xaa^9\x0f}\xf7\xa6\x92\x96\xa7U=\xc1\n\xa3\x1fk\x14\xf9p\x1a\xe3_\xe2\x8cD\n\xdfMN\xa9\xc0&GC\x875\xa4\x1ce\xb9\xe0\x16\xba\xf4\xae\xbfz\xd2T\x992\xd1\xf0\x85Wh\x10\x93\xed\x9c\xb

In [5]:

import aes128R1CS_z as R1CSz

#import importlib
#importlib.reload( aes128R1CS )

z = R1CSz.get_vec_z(pt,sk,64)

print( R1CSz.aes128R1CS_num_constrains , len(z) )


3656 3320


In [6]:
from aes128R1CS import get_aes128r1cs

################  test   #####################################


mat_a , mat_b , mat_c = get_aes128r1cs( R1CSz.aes128R1CS_num_constrains , len(z) , 64, 128)

In [7]:
print( f"mat_a: {mat_a.n_cols}x{mat_a.n_rows}" )
print( f"mat_b: {mat_b.n_cols}x{mat_b.n_rows}" )
print( f"mat_c: {mat_c.n_cols}x{mat_c.n_rows}" )

mat_a: 3320x3656
mat_b: 3320x3656
mat_c: 3320x3656


In [8]:
def mat_x_vec( mat , vec ) :
    r = [0]*mat.n_rows
    for i,e in enumerate(vec):
        if 0 == len(mat.col[i]) : continue
        for idx,term in mat.col[i] :
            r[idx] ^= e if 1==term else cl.gf264_mul( term , e )
    return r

#########  test  ############

Az = mat_x_vec( mat_a , z )
Bz = mat_x_vec( mat_b , z )
Cz = mat_x_vec( mat_c , z )

In [9]:
for i in range(R1CSz.aes128R1CS_num_constrains):
    ci = cl.gf264_mul( Az[i] , Bz[i] ) ^ Cz[i]
    if ci :
        print( f"{i}: fails.  Az[{i}]:{hex(Az[i])} x Bz[{i}]:{hex(Bz[i])} ^ Cz[{i}]:{hex(Cz[i])} = {hex(ci)}" )
        break
